Merging

In [2]:
import pandas as pd

df = pd.read_csv('cleaned_lending_club_data.csv')

##### STEP 10 ##### Loading the other datasets

# Load economic indicators
cpi = pd.read_csv('CPIAUCSL.csv')
fedfunds = pd.read_csv('FEDFUNDS.csv')
gdp = pd.read_csv('GDP.csv')
sentiment = pd.read_csv('UMCSENT.csv')

print(cpi.isnull().sum())
print(fedfunds.isnull().sum())
print(gdp.isnull().sum())
print(sentiment.isnull().sum())

# Load housing data 
zhvi_zip_raw = pd.read_csv('zvhi_zip.csv')
zhvi_state_raw = pd.read_csv('zvhi_state.csv')

print(gdp.isnull().sum().sum())
print(sentiment.isnull().sum().sum())

df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')
df['year_month'] = df['issue_d'].dt.to_period('M').astype(str)
df['zip_3digit'] = df['zip_code'].str[:3].astype(int) * 100







observation_date    0
CPIAUCSL            0
dtype: int64
observation_date    0
FEDFUNDS            0
dtype: int64
observation_date    0
GDP                 0
dtype: int64
observation_date    0
UMCSENT             0
dtype: int64
0
0


In [3]:
# ============================================================================
# STEP 11: Data Preparation and Pre-Merge Feature Engineering
# ============================================================================

# ---------------------------------------------------------------------------
# 1. Prepare Main Dataframe Key Columns
# ---------------------------------------------------------------------------
df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')
df['year_month'] = df['issue_d'].dt.to_period('M') # Keep as Period object for easy merging
df['zip_3digit'] = df['zip_code'].str[:3].astype(int) * 100

# ---------------------------------------------------------------------------
# 2. Process & Engineer Economic Indicators (PRE-MERGE)
# ---------------------------------------------------------------------------
print("Processing Economic Indicators...")

# Standardize Dates
for econ_df, name in [(cpi, 'CPI'), (fedfunds, 'FEDFUNDS'), (gdp, 'GDP'), (sentiment, 'UMCSENT')]:
    econ_df['observation_date'] = pd.to_datetime(econ_df['observation_date'])
    econ_df['year_month'] = econ_df['observation_date'].dt.to_period('M')
    econ_df.rename(columns={econ_df.columns[1]: name}, inplace=True)

# Resample GDP to Monthly (Forward Fill)
gdp = gdp.set_index('observation_date').resample('MS').ffill().reset_index()
gdp['year_month'] = gdp['observation_date'].dt.to_period('M')

# Merge all econ data into one lightweight dataframe
econ_data = cpi[['year_month', 'CPI']]
econ_data = econ_data.merge(fedfunds[['year_month', 'FEDFUNDS']], on='year_month', how='outer')
econ_data = econ_data.merge(gdp[['year_month', 'GDP']], on='year_month', how='outer')
econ_data = econ_data.merge(sentiment[['year_month', 'UMCSENT']], on='year_month', how='outer')

# Sort by time is strictly required for shifting
econ_data = econ_data.sort_values('year_month').reset_index(drop=True)
econ_data = econ_data.ffill().bfill() # Handle any small gaps from outer joins

# --- ENGINEER MACRO FEATURES ---
indicators = ['CPI', 'FEDFUNDS', 'GDP', 'UMCSENT']

for col in indicators:
    # Lags
    for k in [1, 3, 6, 12]:
        econ_data[f'{col}_lag_{k}'] = econ_data[col].shift(k)
    
    # Moving Averages
    for n in [3, 6, 12]:
        econ_data[f'{col}_ma_{n}'] = econ_data[col].rolling(window=n, min_periods=1).mean()
    
    # Rate of Change (Percentage)
    for k in [1, 3, 6, 12]:
        econ_data[f'{col}_pct_change_{k}'] = econ_data[col].pct_change(k) * 100

# Composite Econ Indicators
# 1. Real Interest Rate
econ_data['CPI_inflation_12m'] = econ_data['CPI'].pct_change(12) * 100
econ_data['real_interest_rate'] = econ_data['FEDFUNDS'] - econ_data['CPI_inflation_12m']

# 2. Fed Stance
econ_data['fed_stance'] = econ_data['FEDFUNDS'] - econ_data['FEDFUNDS_ma_12']

# 3. Economic Momentum
econ_data['economic_momentum'] = (econ_data['CPI_pct_change_3'] + 
                                  econ_data['GDP_pct_change_3'] + 
                                  econ_data['UMCSENT_pct_change_3']) / 3

# 4. Economic Stress (Fed rising while Sentiment/GDP falling)
# Note: We need housing change for the full equation, but we calculate partial here
econ_data['economic_stress_partial'] = (econ_data['FEDFUNDS_pct_change_3'] - 
                                        econ_data['UMCSENT_pct_change_3']) / 2

# Fill NaNs generated by shifting (First few rows will be NaN)
econ_data = econ_data.bfill()

# ---------------------------------------------------------------------------
# 3. Process & Engineer Housing Data (PRE-MERGE)
# ---------------------------------------------------------------------------
print("Processing Housing Data...")

# -- Helper function to clean ZHVI data --
def process_zhvi(raw_df, region_col, value_col_name):
    # Identify date columns (YYYY-MM-DD)
    date_cols = [col for col in raw_df.columns if col[0].isdigit()]
    id_vars = [col for col in raw_df.columns if col not in date_cols]
    
    # Melt to long format
    df_long = raw_df.melt(id_vars=id_vars, value_vars=date_cols, 
                          var_name='date', value_name=value_col_name)
    
    df_long['date'] = pd.to_datetime(df_long['date'])
    df_long['year_month'] = df_long['date'].dt.to_period('M')
    
    return df_long

# A. Process ZIP Data
zhvi_zip_long = process_zhvi(zhvi_zip_raw, 'RegionName', 'ZHVI')
zhvi_zip_long['zip_3digit'] = (zhvi_zip_long['RegionName'].astype(int) // 100) * 100

# Aggregate to Zip-3 level
zhvi_zip = zhvi_zip_long.groupby(['zip_3digit', 'year_month'])['ZHVI'].mean().reset_index()

# Sort for time series operations
zhvi_zip = zhvi_zip.sort_values(['zip_3digit', 'year_month'])

# --- ENGINEER HOUSING FEATURES (ZIP) ---
# Group by Zip to ensure shifts don't bleed across regions
g = zhvi_zip.groupby('zip_3digit')['ZHVI']

for k in [1, 3, 6, 12]:
    zhvi_zip[f'ZHVI_pct_change_{k}'] = g.pct_change(k) * 100
    zhvi_zip[f'ZHVI_momentum_{k}'] = zhvi_zip[f'ZHVI_pct_change_{k}'] # Alias
    zhvi_zip[f'ZHVI_lag_{k}'] = g.shift(k)

for n in [3, 6, 12]:
    zhvi_zip[f'ZHVI_ma_{n}'] = g.rolling(window=n, min_periods=1).mean().reset_index(0, drop=True)

# Volatility & Trend
zhvi_zip['ZHVI_return'] = g.pct_change()
zhvi_zip['housing_volatility'] = zhvi_zip.groupby('zip_3digit')['ZHVI_return'].transform(
    lambda x: x.rolling(window=6, min_periods=1).std()
)
zhvi_zip['housing_trend_signal'] = (zhvi_zip['ZHVI'] > zhvi_zip['ZHVI_ma_12']).astype(int)

# B. Process State Data (Fallback)
zhvi_state_long = process_zhvi(zhvi_state_raw, 'RegionName', 'ZHVI_state')
zhvi_state = zhvi_state_long.rename(columns={'RegionName': 'state'})[['state', 'year_month', 'ZHVI_state']]

# ---------------------------------------------------------------------------
# 4. JOINS: Merge Enriched Data into Main DataFrame
# ---------------------------------------------------------------------------
print("Merging data...")

# Join 1: Economic Data
df = df.merge(econ_data, on='year_month', how='left')

# Join 2: ZIP Housing Data
df = df.merge(zhvi_zip, on=['zip_3digit', 'year_month'], how='left')

# Join 3: State Housing Data (Fallback)
df = df.merge(zhvi_state, left_on=['addr_state', 'year_month'], 
              right_on=['state', 'year_month'], how='left')

# Apply Fallback: If Zip ZHVI is missing, use State ZHVI
df['ZHVI'] = df['ZHVI'].fillna(df['ZHVI_state'])



Processing Economic Indicators...
Processing Housing Data...


C:\Users\sagro\AppData\Local\Temp\ipykernel_75428\3288331708.py:109: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  zhvi_zip[f'ZHVI_pct_change_{k}'] = g.pct_change(k) * 100
C:\Users\sagro\AppData\Local\Temp\ipykernel_75428\3288331708.py:109: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  zhvi_zip[f'ZHVI_pct_change_{k}'] = g.pct_change(k) * 100
C:\Users\sagro\AppData\Local\Temp\ipykernel_75428\3288331708.py:109: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values 

Merging data...


In [4]:
# ============================================================================
# STEP 12: Interaction Feature Engineering (Post-Merge)
# ============================================================================
print("Engineering Interaction Features...")

# 1. Finish Economic Stress (requires merged Housing data)
# Note: We check if column exists to handle potential NaNs from merge
df['economic_stress'] = (df['economic_stress_partial'] * 2 - df['ZHVI_pct_change_3']) / 3

# 2. Interest Rate Spread
df['interest_rate_spread'] = df['int_rate'] - df['FEDFUNDS']

# 3. Debt-to-Income Adjusted
# Assume monthly housing payment proxy is 0.4% of home value
df['estimated_housing_payment'] = 0.004 * df['ZHVI']
df['dti_adjusted'] = df['dti'] + (12 * df['estimated_housing_payment'] / df['annual_inc']) * 100

# 4. LTV Proxy (Loan Amount / Home Value)
df['ltv_proxy'] = df['loan_amnt'] / df['ZHVI']

# 5. Composite Risk Score
df['composite_risk_score'] = (df['dti']/100 + 
                              df['delinq_2yrs']/10 + 
                              df['interest_rate_spread']/20) / 3

# 6. Credit History Length
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y')
df['credit_history_length_months'] = ((df['issue_d'] - df['earliest_cr_line']).dt.days / 30.44).astype(int)

Engineering Interaction Features...


In [5]:
# ============================================================================
# STEP 13: Handle Missing Values
# ============================================================================
print("Step 13: Handling Missing Values...")

# ---------------------------------------------------------------------------
# 1. Create Missing Indicators
# ---------------------------------------------------------------------------
# We flag these because a missing ZHVI often indicates a rural area
# or a non-standard housing market, which is predictive of risk.
zhvi_cols = ['ZHVI', 'ZHVI_lag_1', 'ZHVI_lag_3', 'ZHVI_lag_6', 'ZHVI_lag_12']
for col in zhvi_cols:
    if col in df.columns:
        # Create boolean flag (1 if missing, 0 otherwise)
        df[f'{col}_missing'] = df[col].isnull().astype(int)

# ---------------------------------------------------------------------------
# 2. Median Imputation
# ---------------------------------------------------------------------------
# Rationale: We already used State-level fallbacks in Step 11. 
# The remaining gaps (<0.1% of data) are edge cases. 
# Complex time-series imputation here is computationally expensive and 
# offers diminishing returns over median imputation.

numeric_cols = df.select_dtypes(include=['number']).columns
cols_with_missing = [col for col in numeric_cols if df[col].isnull().any()]

if cols_with_missing:
    print(f"Imputing {len(cols_with_missing)} columns with median values...")
    
    # Calculate medians once (efficent)
    fill_values = df[cols_with_missing].median()
    
    # Log what we are filling for sanity check
    for col, val in fill_values.items():
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            print(f"  - {col}: filled {missing_count} rows with {val:.4f}")
            
    # Apply fill
    df[cols_with_missing] = df[cols_with_missing].fillna(fill_values)

# ---------------------------------------------------------------------------
# 3. Final Verification
# ---------------------------------------------------------------------------
print("\n=== Post-Imputation Check ===")
print(f"Final dataset shape: {df.shape}")
print(f"Total missing values: {df.isnull().sum().sum()}")

# Sanity Check
if df.isnull().sum().sum() > 0:
    print("WARNING: Remaining missing values found:")
    print(df.isnull().sum()[df.isnull().sum() > 0])
else:
    print("SUCCESS: No missing values remaining.")

# ---------------------------------------------------------------------------
# 4. Save Final Dataset (Optional but recommended)
# ---------------------------------------------------------------------------
# df.to_csv('data/lc_data_with_economic_features.csv', index=False)

Step 13: Handling Missing Values...
Imputing 24 columns with median values...
  - ZHVI: filled 1688 rows with 215634.8132
  - ZHVI_pct_change_1: filled 1726 rows with 0.4676
  - ZHVI_momentum_1: filled 1726 rows with 0.4676
  - ZHVI_lag_1: filled 1727 rows with 214510.8077
  - ZHVI_pct_change_3: filled 1838 rows with 1.3953
  - ZHVI_momentum_3: filled 1838 rows with 1.3953
  - ZHVI_lag_3: filled 1840 rows with 212244.5878
  - ZHVI_pct_change_6: filled 1984 rows with 2.7955
  - ZHVI_momentum_6: filled 1984 rows with 2.7955
  - ZHVI_lag_6: filled 1992 rows with 209190.7112
  - ZHVI_pct_change_12: filled 2319 rows with 5.4781
  - ZHVI_momentum_12: filled 2319 rows with 5.4781
  - ZHVI_lag_12: filled 2322 rows with 203486.6362
  - ZHVI_ma_3: filled 1688 rows with 214475.1541
  - ZHVI_ma_6: filled 1687 rows with 212743.9710
  - ZHVI_ma_12: filled 1687 rows with 209758.1275
  - ZHVI_return: filled 1726 rows with 0.0047
  - housing_volatility: filled 1790 rows with 0.0017
  - housing_trend_si

In [6]:
# ============================================================================
# SAFETY CHECK: Dropping remaining columns with missing values
# ============================================================================

# 1. Identify columns that still have missing values
cols_to_drop = df.columns[df.isnull().any()].tolist()
print(f"Remaining empty columns detected: {cols_to_drop}")

# 2. Drop them
if cols_to_drop:
    df = df.drop(columns=cols_to_drop)
    print(f"{len(cols_to_drop)} columns dropped.")
else:
    print("No remaining empty columns.")

# 3. Final check (Must print 0)
print(f"Total missing values FINAL: {df.isnull().sum().sum()}")

Remaining empty columns detected: ['state', 'ZHVI_state']
2 columns dropped.
Total missing values FINAL: 0


In [7]:
# ============================================================================
# STEP 14: Final Cleanup and Export
# ============================================================================

# 1. Remove Duplicate Features
# ----------------------------------------------------------------------------
# We created 'ZHVI_momentum' as an alias for 'ZHVI_pct_change' in Step 11.
# We will keep 'momentum' as it sounds more descriptive for a risk feature.
duplicate_cols = [f'ZHVI_pct_change_{k}' for k in [1, 3, 6, 12]]
cols_to_drop_dup = [col for col in duplicate_cols if col in df.columns]

if cols_to_drop_dup:
    df = df.drop(columns=cols_to_drop_dup)
    print(f"Removed {len(cols_to_drop_dup)} duplicate 'pct_change' columns (kept 'momentum').")

# 2. Remove Structural/Temporal Helper Columns
# ----------------------------------------------------------------------------
# We drop columns used for joining (zip_3digit, year_month) 
# and raw location identifiers (zip_code, addr_state).
#
# NOTE: We keep 'issue_d' for the Train/Test split (Time-based splitting).
structural_cols = [
    'year_month', 
    'zip_3digit', 
    'zip_code', 
    'addr_state', 
    'state',
    'ZHVI_state'
    'earliest_cr_line' # We already engineered 'credit_history_length_months'
]

cols_to_drop_struct = [col for col in structural_cols if col in df.columns]
if cols_to_drop_struct:
    df = df.drop(columns=cols_to_drop_struct)
    print(f"Removed {len(cols_to_drop_struct)} structural helper columns.")

# 3. Final Data Check
# ----------------------------------------------------------------------------
print(f"\n{'='*60}")
print(f"FINAL DATASET READY FOR SPLITTING")
print(f"{'='*60}")
print(f"Rows: {df.shape[0]:,}")
print(f"Features: {df.shape[1]}")
print(f"Missing values: {df.isnull().sum().sum()}")

# Check if 'issue_d' is present (Required for valid backtesting)
if 'issue_d' in df.columns:
    print(f"Date Range: {df['issue_d'].min().date()} to {df['issue_d'].max().date()}")
    print("(NOTE: 'issue_d' preserved for time-based train/test splitting)")
else:
    print("WARNING: 'issue_d' was dropped. Ensure you have already split your data!")

print(f"{'='*60}")

# 4. Save to Parquet (Better than CSV for large data types)
# ----------------------------------------------------------------------------
# df.to_parquet('data/lc_final_features.parquet')
# print("Dataset saved to 'data/lc_final_features.parquet'")

Removed 4 duplicate 'pct_change' columns (kept 'momentum').
Removed 4 structural helper columns.

FINAL DATASET READY FOR SPLITTING
Rows: 1,306,387
Features: 151
Missing values: 0
Date Range: 2007-06-01 to 2018-12-01
(NOTE: 'issue_d' preserved for time-based train/test splitting)


In [8]:
#!pip install fastparquet

In [9]:
# ============================================================================
# 4. Save Final Dataset
# ============================================================================

df.to_parquet(
    r'C:\ESILV A4\ESILV A4 DIA\Machine Learning\Projet ML\final_dataset_enhanced.parquet',
    index=False,
    engine='fastparquet' 
)
print(" Dataset sauvegardé avec succès via fastparquet")


 Dataset sauvegardé avec succès via fastparquet


In [22]:
# ============================================================================
# STEP 15: Correlation Heatmap
# ============================================================================

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

print("\n=== Generating Correlation Heatmap ===")

# Get numerical columns (exclude target)
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'loan_status_binary' in numerical_cols:
    numerical_cols.remove('loan_status_binary')

print(f"Calculating correlations for {len(numerical_cols)} features...")

# Calculate correlation matrix
corr_matrix = df[numerical_cols].corr()

# Filter to features with at least one high correlation (to reduce clutter)
THRESHOLD = 0.7
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
high_corr_features = set()

for col in upper_tri.columns:
    if any(abs(upper_tri[col]) > THRESHOLD):
        high_corr_features.add(col)

high_corr_features = sorted(list(high_corr_features))
print(f"Found {len(high_corr_features)} features with |correlation| > {THRESHOLD}")

# Create heatmap
if len(high_corr_features) > 0:
    filtered_corr = corr_matrix.loc[high_corr_features, high_corr_features]

    plt.figure(figsize=(max(16, len(high_corr_features)*0.5), max(14, len(high_corr_features)*0.4)))
    sns.heatmap(
        filtered_corr,
        cmap='coolwarm',
        center=0,
        vmin=-1,
        vmax=1,
        annot=False,
        linewidths=0.5,
        square=True,
        cbar_kws={'label': 'Correlation Coefficient'}
    )
    plt.title(f'Feature Correlation Heatmap (|r| > {THRESHOLD})', fontsize=16, fontweight='bold')
    plt.xticks(rotation=45, ha='right', fontsize=8)
    plt.yticks(rotation=0, fontsize=8)
    plt.tight_layout()
    plt.savefig('C:\\ESILV A4\\ESILV A4 DIA\\Machine Learning\\Projet ML\\correlation_heatmap.png', dpi=300, bbox_inches='tight')
    print(f"✓ Saved heatmap to: ML/correlation_heatmap.png")
    plt.close()
else:
    print("No features with high correlation found")

print("="*60)














=== Generating Correlation Heatmap ===
Calculating correlations for 148 features...
Found 82 features with |correlation| > 0.7
✓ Saved heatmap to: ML/correlation_heatmap.png


In [23]:
# ============================================================================
# STEP 16: Data Validation (Check for inf/extreme values)
# ============================================================================

print("\n=== Data Validation ===")

# Check for inf values
inf_counts = np.isinf(df.select_dtypes(include=[np.number])).sum()
inf_cols = inf_counts[inf_counts > 0]

if len(inf_cols) > 0:
    print(f"\n⚠️  Found inf values in {len(inf_cols)} columns:")
    for col, count in inf_cols.items():
        print(f"  {col}: {count} inf values")

    # Replace inf with NaN, then use median imputation
    print("\nReplacing inf values with median...")
    for col in inf_cols.index:
        # Replace inf with NaN
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)
        # Fill with median
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)
        print(f"  {col}: replaced with median {median_val:.2f}")
else:
    print("✓ No inf values found")

# Verify no more inf/nan values
assert not df.select_dtypes(include=[np.number]).isin([np.inf, -np.inf]).any().any(), "Still have inf values!"
assert not df.isnull().any().any(), "Still have NaN values!"
print("✓ Data validation passed")



=== Data Validation ===

⚠️  Found inf values in 1 columns:
  dti_adjusted: 302 inf values

Replacing inf values with median...
  dti_adjusted: replaced with median 36.63
✓ Data validation passed
